In [ ]:
'''
#Make Test catalog: Lunar data

import os
import pandas as pd

main_path = 'data/mars/test/data/'

list_dfs = []

for root, dirs, files in os.walk(main_path):
    for file in files:
        if file.endswith('.csv'):
            trackfile = os.path.join(root, file)
            df = pd.read_csv(trackfile)
            df['filename'] = file
            list_dfs.append(df)

#Concat all Dataframes
df_concat = pd.concat(list_dfs, ignore_index=True)

#Save
df_concat.to_csv('data/mars/test/data/mars.csv', index=False)

print('Your new CSV with all Test data CSV is ready!')
'''

In [9]:
import pandas as pd
from obspy import read
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import find_peaks
from scipy import stats
import numpy as np

In [8]:
data_mars = pd.read_csv('data/mars/test/data/mars.csv')
data_mars.head()

,time(%Y-%m-%dT%H:%M:%S.%f),rel_time(sec),velocity(c/s),filename
0,2019-05-23T02:00:00.032000,0.00,-0.000000,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv
1,2019-05-23T02:00:00.082000,0.05,0.000199,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv
2,2019-05-23T02:00:00.132000,0.10,-0.001630,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv
3,2019-05-23T02:00:00.182000,0.15,-0.000875,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv
4,2019-05-23T02:00:00.232000,0.20,-0.006137,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv


In [14]:
scaler = MinMaxScaler()
data_mars['velocity(m/s)'] = scaler.fit_transform(data_mars[['velocity(m/s)']])
window_size = 10  # Definir o tamanho da janela de suavização
data_mars['velocity_smooth'] = data_mars['velocity(m/s)'].rolling(window=window_size).mean()

peaks, _ = find_peaks(data_mars['velocity_smooth'], height=0.01)  
data_mars['is_sismo'] = 0
data_mars.loc[peaks, 'is_sismo'] = 1  

data_mars['log_velocity'] = np.log1p(data_mars['velocity(m/s)'].abs()) 

data_mars['z_score'] = np.abs(stats.zscore(data_mars['velocity(m/s)']))
sismos_mars = data_mars[data_mars['z_score'] > 0.1]  #Considering z-score = 0.1
sismos_mars

,time(%Y-%m-%dT%H:%M:%S.%f),rel_time(sec),velocity(c/s),filename,velocity(m/s),is_sismo,velocity_smooth,log_velocity,z_score
465346,2022-05-04T23:27:47.348000,1667.30,7185.175293,XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.csv,0.552114,0,0.550193,0.439618,0.174666
465347,2022-05-04T23:27:47.398000,1667.35,8858.173828,XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.csv,0.552579,0,0.550469,0.439917,0.215332
465348,2022-05-04T23:27:47.448000,1667.40,5982.173340,XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.csv,0.551779,1,0.550633,0.439402,0.145424
465350,2022-05-04T23:27:47.548000,1667.50,-8879.829102,XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.csv,0.547650,0,0.550322,0.436737,0.215832
465351,2022-05-04T23:27:47.598000,1667.55,-14187.830078,XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.csv,0.546175,0,0.549959,0.435784,0.344856
...,...,...,...,...,...,...,...,...,...
568638,2021-12-24T22:53:51.991000,3231.95,4255.402344,XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv,0.551300,0,0.550275,0.439093,0.103451
568812,2021-12-24T22:54:00.691000,3240.65,4394.421387,XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv,0.551338,0,0.550325,0.439118,0.106830
568820,2021-12-24T22:54:01.091000,3241.05,4409.835938,XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv,0.551342,0,0.550409,0.439121,0.107204
568821,2021-12-24T22:54:01.141000,3241.10,4739.887695,XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv,0.551434,1,0.550468,0.439180,0.115227


In [16]:
# Data manipulation
sismos_mars.loc[:, 'rel_time(sec)'] = data_mars['rel_time(sec)'].astype(float).round(0).astype(int)
sismos_mars.loc[:, 'velocity(m/s)'] = data_mars['velocity(m/s)'].astype(float).round(0).astype(int)

In [17]:
np.float = np
np.int = np

In [ ]:
import numpy as np
import pandas as pd
from skmultiflow.data import DataStream
from skmultiflow.meta import AdaptiveRandomForestClassifier
from skmultiflow.evaluation import EvaluatePrequential
from scipy.signal import find_peaks
from sklearn.preprocessing import MinMaxScaler
from scipy import stats


X = sismos_mars[['rel_time(sec)', 'velocity(m/s)']].values
y = sismos_mars['is_sismo'].values  

stream = DataStream(X, y)

model = AdaptiveRandomForestClassifier()

evaluator = EvaluatePrequential(
    pretrain_size=100,  
    max_time=1000, 
    max_samples=len(X),  
    batch_size=1,  
    n_wait=100,  
    show_plot=False,  
    metrics=['accuracy', 'precision', 'recall', 'f1']  
)

evaluator.evaluate(stream=stream, model=[model], model_names=['ARF'])

evaluation_summary = evaluator.get_mean_measurements()
print(evaluation_summary)

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 100 sample(s).
Evaluating...
 ##########---------- [50%] [391.59s]